In [32]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
# convert json string into dictionary
from ast import literal_eval

In [3]:
metaData = pd.read_csv('movies_metadata.csv')
metaData.head()

C:\Users\Ahmed\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [4]:
print(f'There are {metaData.shape[0]} rows and {metaData.shape[1]} Columns')

There are 45466 rows and 24 Columns


In [5]:
print(metaData.columns)

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')


## Simple Recommendation System

In [6]:
# let's Calculate the mean of vote average 
C = metaData['vote_average'].mean()
C

5.618207215133889

In [7]:
M = metaData['vote_count'].quantile(0.90)
M

160.0

In [8]:
# get the top 10% of movies based on the vote_count
qMovies = metaData[metaData['vote_count'] >= M]
qMovies.shape
print(f'Original data shape is {metaData.shape} \n')
print(f'10% of movies based on the vote_count shape is {qMovies.shape}')

Original data shape is (45466, 24) 

10% of movies based on the vote_count shape is (4555, 24)


In [9]:
def weighted_rating(x, m=M, C=C):
    v = x['vote_count']
    R = x['vote_average']
    # Calculation based on the IMDB formula
    return (v/(v+m) * R) + (m/(m+v) * C)

In [10]:
qMovies['score'] = qMovies.apply(lambda x:weighted_rating(x), axis= 1)

C:\Users\Ahmed\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [11]:
qMovies = qMovies.sort_values(by= 'score', ascending= False)
qMovies[['title', 'vote_count', 'vote_average', 'score']].head(15)

,title,vote_count,vote_average,score
314,The Shawshank Redemption,8358.0,8.5,8.445869
834,The Godfather,6024.0,8.5,8.425439
10309,Dilwale Dulhania Le Jayenge,661.0,9.1,8.421453
12481,The Dark Knight,12269.0,8.3,8.265477
2843,Fight Club,9678.0,8.3,8.256385
292,Pulp Fiction,8670.0,8.3,8.251406
522,Schindler's List,4436.0,8.3,8.206639
23673,Whiplash,4376.0,8.3,8.205404
5481,Spirited Away,3968.0,8.3,8.196055
2211,Life Is Beautiful,3643.0,8.3,8.187171


## Content-Based Recommender

In [12]:
metaData['overview'].head()

0    Led by Woody, Andy's toys live happily in his ...
1    When siblings Judy and Peter discover an encha...
2    A family wedding reignites the ancient feud be...
3    Cheated on, mistreated and stepped on, the wom...
4    Just when George Banks has recovered from his ...
Name: overview, dtype: object

In [13]:
print(f"count of null overviews is {metaData['overview'].isnull().sum()}")
# Replace Nan values with an empty string
metaData['overview'].fillna('',inplace=True)

count of null overviews is 954


In [14]:
tfidf = TfidfVectorizer(stop_words="english")
tfidfMatrix = tfidf.fit_transform(metaData['overview'])
tfidfMatrix.shape

(45466, 75827)

In [15]:
tfidf.get_feature_names()[6000:6010]

C:\Users\Ahmed\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['barracuda',
 'barrage',
 'barraged',
 'barrages',
 'barratier',
 'barratt',
 'barraza',
 'barre',
 'barreaux',
 'barred']

In [16]:
tfidfMatrix = tfidfMatrix[:30000]
cosineSimilarity = linear_kernel(tfidfMatrix,tfidfMatrix)
cosineSimilarity.shape

(30000, 30000)

In [17]:
cosineSimilarity[0][:5]

array([1.        , 0.01504121, 0.        , 0.        , 0.        ])

In [18]:
indices = pd.Series(metaData.index , index = metaData['title']).drop_duplicates()

In [19]:
indices[:10]

title
Toy Story                      0
Jumanji                        1
Grumpier Old Men               2
Waiting to Exhale              3
Father of the Bride Part II    4
Heat                           5
Sabrina                        6
Tom and Huck                   7
Sudden Death                   8
GoldenEye                      9
dtype: int64

In [20]:
def getRecommendation(title, cosine_sim= cosineSimilarity):
  # getting the index of the movie that matches the title
  idx = indices[title]
  #print(idx)
  sim_scores = list(enumerate(cosine_sim[idx]))
  #print(sim_scores)
  sim_scores = sorted(sim_scores, key= lambda x : x[1], reverse= True)
  #print(sim_scores)
  sim_scores = sim_scores[1:11]

  movieIndices = [i[0] for i in sim_scores]

  return metaData.iloc[movieIndices]['title']
getRecommendation('Toy Story')

15348                    Toy Story 3
2997                     Toy Story 2
10301         The 40 Year Old Virgin
24523                      Small Fry
23843    Andy Hardy's Blonde Trouble
29202                     Hot Splash
8327                       The Champ
27206     Life Begins for Andy Hardy
1071           Rebel Without a Cause
1932                       Condorman
Name: title, dtype: object

In [21]:
getRecommendation('The Dark Knight Rises')

12481                                      The Dark Knight
150                                         Batman Forever
1328                                        Batman Returns
15511                           Batman: Under the Red Hood
585                                                 Batman
21194    Batman Unmasked: The Psychology of the Dark Kn...
9230                    Batman Beyond: Return of the Joker
18035                                     Batman: Year One
19792              Batman: The Dark Knight Returns, Part 1
3095                          Batman: Mask of the Phantasm
Name: title, dtype: object

In [22]:
getRecommendation('The Godfather')

1178      The Godfather: Part II
1914     The Godfather: Part III
23126                 Blood Ties
11297           Household Saints
10821                   Election
17729          Short Sharp Shock
26293         Beck 28 - Familjen
8653                Violent City
13177               I Am the Law
26660                   Suddenly
Name: title, dtype: object

In [23]:
credits = pd.read_csv('credits.csv')
keywords = pd.read_csv('keywords.csv')

In [24]:
credits.head()

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


In [25]:
keywords.head()

,id,keywords
0,862,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,8844,"[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,15602,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,31357,"[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,11862,"[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."


In [26]:
metaData = metaData.drop([19730, 29503, 35587])

metaData['id'] = metaData['id'].astype(int)
keywords['id'] = keywords['id'].astype(int)
credits['id'] = credits['id'].astype(int)

metaData = metaData.merge(keywords,on='id')
metaData = metaData.merge(credits,on='id')

metaData.head(2)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,spoken_languages,status,tagline,title,video,vote_average,vote_count,keywords,cast,crew
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,...","[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de..."
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,"[{'id': 10090, 'name': 'board game'}, {'id': 1...","[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de..."


In [27]:
metaData.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count', 'keywords', 'cast', 'crew'],
      dtype='object')

In [31]:
print(metaData['crew'][0])

[{'credit_id': '52fe4284c3a36847f8024f49', 'department': 'Directing', 'gender': 2, 'id': 7879, 'job': 'Director', 'name': 'John Lasseter', 'profile_path': '/7EdqiNbr4FRjIhKHyPPdFfEEEFG.jpg'}, {'credit_id': '52fe4284c3a36847f8024f4f', 'department': 'Writing', 'gender': 2, 'id': 12891, 'job': 'Screenplay', 'name': 'Joss Whedon', 'profile_path': '/dTiVsuaTVTeGmvkhcyJvKp2A5kr.jpg'}, {'credit_id': '52fe4284c3a36847f8024f55', 'department': 'Writing', 'gender': 2, 'id': 7, 'job': 'Screenplay', 'name': 'Andrew Stanton', 'profile_path': '/pvQWsu0qc8JFQhMVJkTHuexUAa1.jpg'}, {'credit_id': '52fe4284c3a36847f8024f5b', 'department': 'Writing', 'gender': 2, 'id': 12892, 'job': 'Screenplay', 'name': 'Joel Cohen', 'profile_path': '/dAubAiZcvKFbboWlj7oXOkZnTSu.jpg'}, {'credit_id': '52fe4284c3a36847f8024f61', 'department': 'Writing', 'gender': 0, 'id': 12893, 'job': 'Screenplay', 'name': 'Alec Sokolow', 'profile_path': '/v79vlRYi94BZUQnkkyznbGUZLjT.jpg'}, {'credit_id': '52fe4284c3a36847f8024f67', 'depart

In [34]:
featuresJson = ['cast', 'crew', 'keywords', 'genres']
for feature in featuresJson:
    metaData[feature] = metaData[feature].apply(literal_eval)

In [42]:
print(metaData['crew'][2][0]['job'])

Director


In [43]:
def getDirector(json):
    for dic in json:
        if dic['job'] == "Director":
            return dic['name']
    return np.nan    

In [45]:
metaData['director'] = metaData['crew'].apply(getDirector)

In [49]:
print(metaData['genres'][0])

[{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]


In [60]:
print(metaData['cast'][3])

[{'cast_id': 1, 'character': "Savannah 'Vannah' Jackson", 'credit_id': '52fe44779251416c91011aad', 'gender': 1, 'id': 8851, 'name': 'Whitney Houston', 'order': 0, 'profile_path': '/69ouDnXnmklYPr4sMJXWKYz81AL.jpg'}, {'cast_id': 2, 'character': "Bernadine 'Bernie' Harris", 'credit_id': '52fe44779251416c91011ab1', 'gender': 1, 'id': 9780, 'name': 'Angela Bassett', 'order': 1, 'profile_path': '/tHkgSzhEuJKp5hqp0DZLad8HNZ9.jpg'}, {'cast_id': 3, 'character': "Gloria 'Glo' Matthews", 'credit_id': '52fe44779251416c91011ab5', 'gender': 1, 'id': 18284, 'name': 'Loretta Devine', 'order': 2, 'profile_path': '/zLQFwQTFtHkb8sbFdkPNamFI7jv.jpg'}, {'cast_id': 4, 'character': 'Robin Stokes', 'credit_id': '52fe44779251416c91011ab9', 'gender': 1, 'id': 51359, 'name': 'Lela Rochon', 'order': 3, 'profile_path': '/9DBu3r5O4fBosSS4FnSzFCVpm0O.jpg'}, {'cast_id': 5, 'character': 'Marvin King', 'credit_id': '52fe44779251416c91011abd', 'gender': 2, 'id': 66804, 'name': 'Gregory Hines', 'order': 4, 'profile_path

In [53]:
print(metaData['keywords'][0])

[{'id': 931, 'name': 'jealousy'}, {'id': 4290, 'name': 'toy'}, {'id': 5202, 'name': 'boy'}, {'id': 6054, 'name': 'friendship'}, {'id': 9713, 'name': 'friends'}, {'id': 9823, 'name': 'rivalry'}, {'id': 165503, 'name': 'boy next door'}, {'id': 170722, 'name': 'new toy'}, {'id': 187065, 'name': 'toy comes to life'}]


In [63]:
def getList(x):
    if isinstance(x,list):
        names = [i['name'] for i in x]
        # check if name's length is more than 3 exist 
        if len(names) > 3:
            names = names[:3]
        return names
    return []

In [64]:
features = ['cast','keywords','genres']

for feature in features:
    metaData[feature] = metaData[feature].apply(getList)

In [65]:
metaData['genres'][0]

['Animation', 'Comedy', 'Family']

In [66]:
def textProcessing(x):
    if isinstance(x,list):
        return [i.replace(" ","").lower() for i in x]
    if isinstance(x,str):
        return x.replace(" ","").lower()
    else:
        return ""

In [69]:
textProcessing(['ahmed basem','mohamed basem'])

['ahmedbasem', 'mohamedbasem']

In [72]:
features = ['cast', 'director', 'keywords', 'genres']
for feature in features:
    metaData[feature] = metaData[feature].apply(textProcessing)

In [73]:
def getSoup(x):
    return " ".join(x['keywords']) + " " + " ".join(x['cast']) + " " \
    + " ".join(x['director']) + " " + " ".join(x['genres'])

In [74]:
metaData['soup'] = metaData.apply(getSoup,axis=1)

In [77]:
metaData[['soup','keywords', 'cast', 'director', 'genres']].head(3)

,soup,keywords,cast,director,genres
0,jealousy toy boy tomhanks timallen donrickles ...,"[jealousy, toy, boy]","[tomhanks, timallen, donrickles]",johnlasseter,"[animation, comedy, family]"
1,boardgame disappearance basedonchildren'sbook ...,"[boardgame, disappearance, basedonchildren'sbook]","[robinwilliams, jonathanhyde, kirstendunst]",joejohnston,"[adventure, fantasy, family]"
2,fishing bestfriend duringcreditsstinger walter...,"[fishing, bestfriend, duringcreditsstinger]","[waltermatthau, jacklemmon, ann-margret]",howarddeutch,"[romance, comedy]"


In [78]:
from sklearn.feature_extraction.text import CountVectorizer

In [79]:
vectorizer = CountVectorizer(stop_words='english')
soupMatrix = vectorizer.fit_transform(metaData['soup'])

In [80]:
soupMatrix.shape

(46628, 58204)

20000 for prevent allocate more memory than is available. It has restarted.

In [83]:
soupMatrix = soupMatrix[:20000]

In [84]:
from sklearn.metrics.pairwise import cosine_similarity

In [85]:
soupSimilarity = cosine_similarity(soupMatrix,soupMatrix)

In [90]:
#Reset index of our main DataFrame and construct reverse mapping as before
metaData = metaData.reset_index()
indices = pd.Series(index=metaData['title'],data=metaData.index).drop_duplicates()

In [88]:
getRecommendation("The Dark Knight Rises",soupSimilarity)

12589      The Dark Knight
10210        Batman Begins
9311                Shiner
9874       Amongst Friends
7772              Mitchell
516      Romeo Is Bleeding
4520       An Innocent Man
5048        State Property
6095              Lockdown
6758      Once in the Life
Name: title, dtype: object

In [89]:
getRecommendation('The Godfather', soupSimilarity)


1934            The Godfather: Part III
8001     The Night of the Following Day
18261                 The Son of No One
7772                           Mitchell
18940                         Last Exit
5309                        The Gambler
11733          The Consequences of Love
16782                     The Organizer
5                                  Heat
426                       Carlito's Way
Name: title, dtype: object

## Collaborative Filtering Recommender